In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import sys
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
import ast   
import re
import matplotlib.patches as patches
import matplotlib.image as mpimg  
import matplotlib.colors as mcolors

# import DeepEye analysis functions
from deepeye_analysis_package.getFixations import batch_extract_fixations
from deepeye_analysis_package.preprocessing import getFixationLatency, handle_carryover_fixations_and_merge, addAOI
from deepeye_analysis_package.plotting import plot2d

### Main part

In [3]:
# Determine the environment ('Home' or 'Office') and set the data path accordingly
WHERE = 'Office'  # 'Office' or 'Home'

if WHERE == 'Home':
    path = r'C:/Users/aby600/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/Young/Approved'
else:
    path = r'D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/Young/Approved'

# Define the AOI padding in pixels
PADDING = 0  # padding of AOI on each side, used in plot2d() and addAOI()

# Helper function to create a directory if it doesn't exist
def create_directory_if_not_exists(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print(f"Directory '{directory_path}' was created.")
    else:
        print(f"Directory '{directory_path}' already exists.")

# Define data analysis directories and create them if they don't exist yet
path_to_data = os.path.join(path, 'data')
path_to_analysis = os.path.join(path, 'analysis_new')
create_directory_if_not_exists(path_to_analysis)

# Initialize an empty list to hold the processed dataframes
output_dfs = []

# Get all folder names from the data directory
folder_names = [name for name in os.listdir(path_to_data) if os.path.isdir(os.path.join(path_to_data, name))]

# Process each participant's data
for fn in folder_names:
    path_to_file = os.path.join(path_to_data, fn, f'{fn}_record_extra.csv')

    print(f'Processing participant {fn}...')

    try:
        df = pd.read_csv(path_to_file)
    except FileNotFoundError:
        print(f'File does not exist: {path_to_file}')
        continue

    # Filter data to only include rows where the target was presented
    df1 = df[df['event'] == 'target_on'].copy()

    # Add padding, subject ID, and image dimensions to the dataframe
    df1['padding'] = PADDING
    df1['deepeye_id'] = fn
    df1['imageDims'] = [(480, 480)] * len(df1)

    # Add image paths and coordinates to the dataframe
    df1['image_paths'] = df1.apply(lambda row: [row.imageLeft, row.imageRight], axis=1)
    df1['image_coords'] = df1.apply(lambda row: [
        (row.leftX, row.Y, row.imageDims[0], row.imageDims[1]),
        (row.rightX, row.Y, row.imageDims[0], row.imageDims[1])
    ], axis=1)

    # Add bounding boxes and their names to the dataframe
    df1['bboxes'] = df1.apply(lambda row: [
        [row.leftX, row.Y, row.imageDims[0], row.imageDims[1]],
        [row.rightX, row.Y, row.imageDims[0], row.imageDims[1]]
    ], axis=1)
    df1['bboxesNames'] = df1.apply(lambda row: ['left', 'right'], axis=1)

    # Plot 2D fixations without saving the plot
    # plot2d(df1, fn, path_to_analysis, condition='locStudiedImage', save=False)

    # Process the data by applying preprocessing steps
    df1 = getFixationLatency(df1)
    df1 = handle_carryover_fixations_and_merge(df1, max_event_duration=4000)
    df1 = addAOI(df1)

    # Accumulate the processed dataframe for this participant
    output_dfs.append(df1)

# Concatenate all participants' data into one DataFrame
if output_dfs:
    output_df = pd.concat(output_dfs, ignore_index=True)
    output_file = os.path.join(path_to_analysis, 'allSubjects_PV_Young.csv')
    output_df.to_csv(output_file, index=False)
    print(f'Combined data saved to {output_file}')
else:
    print('No data was processed.')


Directory 'D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/Young/Approved\analysis_new' already exists.
Processing participant 2024_01_15_11_44_18...


d:\GitHub\DeepEye_analyze\deepeye_analysis_package\preprocessing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group.FixStartEnd.iloc[0] = 'fix_end_carryover_inserted_start'
d:\GitHub\DeepEye_analyze\deepeye_analysis_package\preprocessing.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group.FixDur.iloc[0] = group.sampTime.iloc[0] - group.targSampTime.iloc[0]
d:\GitHub\DeepEye_analyze\deepeye_analysis_package\preprocessing.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

Processing participant 2024_01_15_12_03_52...


d:\GitHub\DeepEye_analyze\deepeye_analysis_package\preprocessing.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group.FixStartEnd.iloc[-1] = 'fix_start_carryover_inserted_end'
d:\GitHub\DeepEye_analyze\deepeye_analysis_package\preprocessing.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group.FixDur.iloc[-1] = 0
d:\GitHub\DeepEye_analyze\deepeye_analysis_package\preprocessing.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

Processing participant 2024_01_15_14_08_56...


...

Processing participant 2024_01_27_13_26_23...


d:\GitHub\DeepEye_analyze\deepeye_analysis_package\preprocessing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group.FixStartEnd.iloc[0] = 'fix_end_carryover_inserted_start'
d:\GitHub\DeepEye_analyze\deepeye_analysis_package\preprocessing.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group.FixDur.iloc[0] = group.sampTime.iloc[0] - group.targSampTime.iloc[0]
d:\GitHub\DeepEye_analyze\deepeye_analysis_package\preprocessing.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

Processing participant 2024_01_27_14_22_06...


d:\GitHub\DeepEye_analyze\deepeye_analysis_package\preprocessing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group.FixStartEnd.iloc[0] = 'fix_end_carryover_inserted_start'
d:\GitHub\DeepEye_analyze\deepeye_analysis_package\preprocessing.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group.FixDur.iloc[0] = group.sampTime.iloc[0] - group.targSampTime.iloc[0]
d:\GitHub\DeepEye_analyze\deepeye_analysis_package\preprocessing.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

Combined data saved to D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/Young/Approved\analysis_new\allSubjects_PV_Young.csv


## Calculate novelty index and make a new dataframe

In [4]:
# testPhase_df = pd.read_csv(os.path.join(path_to_analysis, 'allSubjects_PV_Young.csv'))

testPhase_df = output_df

# Select only test phase
testPhase_df = testPhase_df[testPhase_df.phase=='test']

# Label the fixations on left or right side
testPhase_df['FixatedNovel'] = np.where(testPhase_df.AOI_stim == testPhase_df.locStudiedImage, 'old', 
                                        np.where(testPhase_df.AOI_stim == 'None', 'None', 'novel'))

novelty_data = []

# Iterate through participants and trials
for (deepeye_id, trialNr), group in testPhase_df.groupby(['deepeye_id','trialNr']):
    
    # Safely compute the proportion of novel fixations (fixCountProp)
    fix_count_total = group.FixatedNovel.count()
    fix_count_novel = group.FixatedNovel[group.FixatedNovel == 'novel'].count()
    novelty_fix_count_prop = fix_count_novel / fix_count_total if fix_count_total != 0 else 0
    
    # Safely compute the proportion of fixation durations (fixDurProp)
    fix_dur_total = group.FixDur.sum()
    fix_dur_novel = group.FixDur[group.FixatedNovel == 'novel'].sum()
    novelty_fix_dur_prop = fix_dur_novel / fix_dur_total if fix_dur_total != 0 else 0
    
    # Append the results to a list
    novelty_data.append([deepeye_id, trialNr, novelty_fix_count_prop, novelty_fix_dur_prop])

# Convert list to DataFrame
novelty_df = pd.DataFrame(novelty_data, columns=['deepeye_id', 'trialNr', 'noveltyIdx_fixCountProp', 'noveltyIdx_fixDurProp'])

# Merge additional data into novelty_df
additional_columns = ['deepeye_id', 'trialNr', 'pp_id', 'imageLeft', 'imageRight', 'locStudiedImage']  # List the columns you want to keep

# Drop duplicates to avoid having repeated rows during merge
testPhase_unique_df = testPhase_df[additional_columns].drop_duplicates(subset=['deepeye_id', 'trialNr'])

# Merge the novelty dataframe with additional information
novelty_df = novelty_df.merge(testPhase_unique_df, on=['deepeye_id', 'trialNr'], how='left')

# Save the output file
novelty_df.to_csv(os.path.join(path_to_analysis, 'allSubjects_NoveltyIndex.csv'), index=False)



C:\Users\Artem\AppData\Local\Temp\ipykernel_8368\3717596633.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testPhase_df['FixatedNovel'] = np.where(testPhase_df.AOI_stim == testPhase_df.locStudiedImage, 'old',


## Summary of novelty index

In [5]:
# filter out excluded participants
# '2024_01_15_14_19_20' was the second time, '2024_01_26_17_16_28' too few frames, '2024_01_15_11_44_18' & '2024_01_26_13_09_05'keep fixating the center, 
novelty_df = novelty_df[~novelty_df['deepeye_id'].isin(['2024_01_15_14_19_20', '2024_01_26_17_16_28', '2024_01_15_11_44_18', '2024_01_26_13_09_05'])]

total_count = novelty_df.groupby(['deepeye_id']).noveltyIdx_fixCountProp.count()
fixCountProp = novelty_df.groupby(['deepeye_id']).noveltyIdx_fixCountProp.mean()
fixDurProp = novelty_df.groupby(['deepeye_id']).noveltyIdx_fixDurProp.mean()

print(total_count)
print(fixCountProp)
print(fixDurProp)
print(fixCountProp.mean())
print(fixDurProp.mean())


deepeye_id
2024_01_15_12_03_52    10
2024_01_15_14_08_56    10
2024_01_15_15_09_08    10
2024_01_15_15_49_33    10
2024_01_15_15_57_52    10
2024_01_15_16_26_53    10
2024_01_15_16_38_59    10
2024_01_15_17_01_45    10
2024_01_15_17_11_36    10
2024_01_15_17_29_15    10
2024_01_15_17_43_23    10
2024_01_15_18_08_09    10
2024_01_26_14_16_01    10
2024_01_26_14_25_48     9
2024_01_26_14_53_12    10
2024_01_26_15_38_25    10
2024_01_27_13_26_23    10
2024_01_27_14_22_06    10
Name: noveltyIdx_fixCountProp, dtype: int64
deepeye_id
2024_01_15_12_03_52    0.600595
2024_01_15_14_08_56    0.722024
2024_01_15_15_09_08    0.594524
2024_01_15_15_49_33    0.648095
2024_01_15_15_57_52    0.565833
2024_01_15_16_26_53    0.613810
2024_01_15_16_38_59    0.469286
2024_01_15_17_01_45    0.333929
2024_01_15_17_11_36    0.515000
2024_01_15_17_29_15    0.693333
2024_01_15_17_43_23    0.241905
2024_01_15_18_08_09    0.695238
2024_01_26_14_16_01    0.673889
2024_01_26_14_25_48    0.453704
2024_01_26_14_53_1